In [1]:
import os
%pwd

'c:\\Users\\pavan\\Desktop\\Fraud-Detection-E2E-ML-project-using-MLflow\\Notebooks'

In [3]:
os.chdir('../')

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pavannagula/Fraud-Detection-E2E-ML-project-using-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pavannagula"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e9ddb8c4d256cae340f50dcb52c6488ecd28235c"

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/pavannagula/Fraud-Detection-E2E-ML-project-using-MLflow.mlflow",
           
        )

        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_model(self, y_true, y_pred):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        
        return accuracy, precision, recall
        

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (accuracy, precision, recall) = self.evaluate_model(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "precision": precision, "recall": recall}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")



In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-25 13:20:04,365: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-25 13:20:04,369: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-25 13:20:04,371: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-25 13:20:04,371: INFO: common: created directory at: artifacts]
[2023-08-25 13:20:04,379: INFO: common: created directory at: artifacts/model_evaluation]
[2023-08-25 13:20:05,537: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'XGBClassifier'.
2023/08/25 13:20:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBClassifier, version 1
Created version '1' of model 'XGBClassifier'.
